In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

# data_key = 'ml-1m/'
# data_key = 'amz_Books/'
# data_key = 'amz_Electronics/'
data_key = 'amz_Movies_and_TV/'
# data_key = 'BX/'
PROCESSED_DATA_ROOT = "/home/sl1471/workspace/experiments/"
target_path = PROCESSED_DATA_ROOT + data_key

### 1. Extract Results from Logs

In [2]:
from utils import extract_results
log_path = target_path + "logs/"
# results = extract_results(log_path, ['n_interest', 'threshold_c'])
# results = extract_results(log_path, ['n_interest'])
results = extract_results(log_path, ['lr', 'l2_coef', 'fair_group_feature', 'fair_lambda', 'n_local_step'], "f2rec_train_and_eval")
print(results.keys())

26it [00:00, 2151.35it/s]

dict_keys([0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 25])


In [3]:
import json
print("Example:")
print(json.dumps(list(results.values())[0], indent = 4))

Example:
{
    "args": "cuda=3, seed=9, train=False, train_and_eval=True, continuous_train=False, eval=False, model_path='/home/sl1471/workspace/experiments/amz_Movies_and_TV/models/f2rec_FairMF_lr0.00001_reg1.0_pairwisebpr_lambda0._gactivity.pkl', loss='pairwisebpr', l2_coef=1.0, emb_size=32, data_file='/home/sl1471/workspace/experiments/amz_Movies_and_TV/tsv_data/', n_worker=4, user_meta_data='/home/sl1471/workspace/experiments/amz_Movies_and_TV/meta_data/user.meta', item_meta_data='/home/sl1471/workspace/experiments/amz_Movies_and_TV/meta_data/item.meta', user_fields_meta_file='/home/sl1471/workspace/experiments/amz_Movies_and_TV/meta_data/user_fields.meta', item_fields_meta_file='/home/sl1471/workspace/experiments/amz_Movies_and_TV/meta_data/item_fields.meta', user_fields_vocab_file='/home/sl1471/workspace/experiments/amz_Movies_and_TV/meta_data/user_fields.vocab', item_fields_vocab_file='/home/sl1471/workspace/experiments/amz_Movies_and_TV/meta_data/item_fields.vocab', n_neg=1, n_

In [4]:
import numpy as np
metrics = []
for m_name in ['HR', 'RECALL', 'P', 'F1', 'NDCG']:
    metrics += [f"{m_name}@{i}" for i in [1,5,10,20,50]]
metrics += ["AUC", "MR", "MRR"]
for m_name in metrics:
    for k, res_dict in results.items():
        if m_name in res_dict:
            res_dict[m_name] = np.mean(res_dict[m_name])

In [5]:
import pandas as pd
df = pd.DataFrame.from_dict(results, orient = 'index')

In [6]:
import os
import datetime
try:
    os.mkdir(target_path + "results/")
except:
    print("Dir exists: '" + target_path + "results/'")
result_file_path = target_path + "results/" + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + ".csv"
df.to_csv(result_file_path, sep = '\t')

Dir exists: '/home/sl1471/workspace/experiments/amz_Movies_and_TV/results/'
